In [1]:
# only do if key not defined
if 'key' not in locals():
    key = !op read "op://private/sfm openai api key/password"
    assert len(key) == 1
    key = key[0]
assert len(key) == 51

In [2]:
import openai
import os
import json
from pprint import pprint
from pydantic import BaseModel

openai.api_key = key

In [3]:
class RunTest(BaseModel):
    include_visual_observation: bool
    agent_test_code: str

In [4]:
def read_test():
    path = r"D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\OpenDoorTest.java"
    with open(path, "r") as f:
        return f.read()

In [5]:
def write_test(content: str):
    path = r"D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\OpenDoorTest.java"
    with open(path, "w") as f:
        f.write(content)

In [6]:
def indent_text(text, spaces=4):
    space_str = " " * spaces
    return "\n".join(f"{space_str}{line}" for line in text.split("\n"))
print(indent_text("a\nb\nc",8))

        a
        b
        c


In [7]:
def with_agent_content(test: str, new_content: str) -> str:
    """
    We want to remove any previous attempts at the test

    // begin agent code
    item.setPos(Vec3.atCenterOf(helper.absolutePos(pressurePlatePos).offset(0,3,0)));
    // end agent code

    should remove the content between the two comments
    """
    region_start = "        // begin agent code"
    region_end = "        // end agent code"
    start = test.find(region_start)
    end = test.find(region_end)
    return test[:start+len(region_start)] + "\n" + new_content + ("\n" if new_content != "" else "") + test[end:]

In [8]:
def get_agent_content(test: str) -> str:
    region_start = "        // begin agent code"
    region_end = "        // end agent code"
    start = test.find(region_start)
    end = test.find(region_end)
    return test[start+len(region_start)+1:end]

# begin hotkey_test

In [9]:
import pyautogui
import pygetwindow as gw
from time import sleep
import pyperclip

In [10]:
def focus_intellij():
    try:
        title = next(x for x in gw.getAllTitles() if "TestChambers.java" in x)
        windows = gw.getWindowsWithTitle(title)
        assert len(windows) > 0, f"Window not found: {title}"
        windows[0].activate()
    except Exception as e:
        print(f"Error focusing window: {e}")

In [11]:
def is_building():
    return pyautogui.pixel(1947, 622) == (95, 173, 101)

In [12]:
def get_build_output():
    # store cursor position
    x, y = pyautogui.position()
    # click and drag from 2890, 638 to 2895, 638 to select text without activating links
    pyautogui.moveTo(2890, 638)
    pyautogui.mouseDown()
    pyautogui.moveTo(2895, 638)
    pyautogui.mouseUp()
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.hotkey('ctrl', 'c')
    # restore cursor position
    pyautogui.moveTo(x, y)
    return pyperclip.paste().replace('\r', '')

In [13]:
def is_happy_build_output(output):
    happy = """
Executing pre-compile tasks…
Running 'before' tasks
Checking sources
Running 'after' tasks
Finished, saving caches…
Executing post-compile tasks…
Finished, saving caches…
    """.strip()
    return output.strip() == happy

In [14]:
def hot_reload():
    print("[Hot Reload] Focusing IntelliJ")
    focus_intellij()
    sleep(0.5)
    print("[Hot Reload] Trigger hot reload")
    pyautogui.hotkey('ctrl', 'alt', 'num0')
    print("[Hot Reload] Wait for build to finish")
    while is_building():
        sleep(0.1)
    sleep(0.5)
    print("[Hot Reload] Copy build output")
    output = get_build_output()
    success = is_happy_build_output(output)
    print(f"[Hot Reload] Build output succeeded: {success}")
    return success, output

# end hotkey_test

In [18]:
print(with_agent_content(read_test(), ''))

package ca.teamdman.sfm.ai;

import ca.teamdman.sfm.SFM;
import ca.teamdman.sfm.SFMGameTestBase;
import net.minecraft.core.BlockPos;
import net.minecraft.gametest.framework.GameTest;
import net.minecraft.gametest.framework.GameTestHelper;
import net.minecraft.world.entity.item.ItemEntity;
import net.minecraft.world.item.ItemStack;
import net.minecraft.world.item.Items;
import net.minecraft.world.level.block.Blocks;
import net.minecraft.world.level.block.DoorBlock;
import net.minecraft.world.phys.Vec3;
import net.neoforged.gametest.GameTestHolder;
import net.neoforged.gametest.PrefixGameTestTemplate;

@GameTestHolder(SFM.MOD_ID)
@PrefixGameTestTemplate(false)
public class OpenDoorTest extends SFMGameTestBase {
    @GameTest(template = "3x4x3")
    public static void open_door(GameTestHelper helper) {
        var pressurePlatePos = new BlockPos(0, 2, 1);
        var redstonePos = new BlockPos(1, 2, 1);
        var doorPos = new BlockPos(2, 2, 1);

        // set the floor to iron blocks


In [15]:
messages = [
    {"role": "system", "content": "The assistant is tasked with solving puzzles in a Minecraft game test environment, similar to the video game _Portal_. The assistant is presented a game test with some code at the beginning and end of the test that the agent can not change. The agent is responsible for replacing the code in the middle of the test to cause the test to succeed."},
    {"role": "user", "content": f"Current test content:\n```java\n{with_agent_content(read_test(), '')}```"},
]

In [16]:
def save_convo(messages):
    content = ""
    for msg in messages:
        content += f"# ~=~ {msg['role']}\n{msg['content']}\n"
        if "function_call" in msg:
            content += f"---\n{msg['function_call']}\n"
    with open("convo.md", "w") as f:
        f.write(content)

In [19]:
response0 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=[
        {
          "name": "run_test",
          "description": "Run the test with the agent code block substituted for the provided content. Requesting a visual observation will slow down the process through a dependency on humans.",
          "parameters": RunTest.model_json_schema()
        },
    ],
    function_call={"name": "run_test"}
)
print(response0)

{
  "id": "chatcmpl-85OuIDnhjVOAnV5MoJg8xm5cooaub",
  "object": "chat.completion",
  "created": 1696298074,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "run_test",
          "arguments": "{\n  \"include_visual_observation\": true,\n  \"agent_test_code\": \"item.setPos(Vec3.atCenterOf(helper.absolutePos(pressurePlatePos).offset(0, 3, 0)));\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 678,
    "completion_tokens": 46,
    "total_tokens": 724
  }
}


In [25]:
run = RunTest(**json.loads(response0.choices[0]["message"]["function_call"]["arguments"]))
print(run.agent_test_code)

item.setPos(Vec3.atCenterOf(helper.absolutePos(pressurePlatePos).offset(0, 3, 0)));


In [20]:
write_test(with_agent_content(read_test(), indent_text(run.agent_test_code, 12)))
success, build_output = hot_reload()
if not success:
    print(build_output)

[Hot Reload] Focusing IntelliJ
Error focusing window: 
[Hot Reload] Trigger hot reload
[Hot Reload] Wait for build to finish
[Hot Reload] Copy build output
[Hot Reload] Build output succeeded: True


# begin runner_comms

In [21]:
from pathlib import Path
msgdir = Path("./messages")
msgdir.mkdir(parents=True, exist_ok=True)

In [22]:
def run_test():
    print("[Run Test] Running test, ensure you ran `/sfm_ai listen` in Minecraft")
    # touch messages/run.txt
    (msgdir / "run.txt").touch()

    from time import sleep
    # wait for the file to have test results appended
    while not (msgdir / "run.txt").stat().st_size > 0:
        sleep(0.1)
    
    # read the file
    with open(msgdir / "run.txt", "r") as f:
        results = f.read()

    archive_content = get_agent_content(read_test()) + "\n\n" + results
    
    # clean up the file by renaming it with the time
    from datetime import datetime
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    (msgdir / f"run_{now}.txt").write_text(archive_content)
    (msgdir / "run.txt").unlink()

    return results

# end runner_comms

In [23]:
if success:
    test_output = run_test()

[Run Test] Running test, ensure you ran `/sfm_ai listen` in Minecraft


In [24]:
test_output

'open_door passed! (544ms)'

In [44]:
fn0 = {
    "role": "function",
    "name": "run_test",
    "content": build_output if not success else test_output,
}
print(fn0)

{'role': 'function', 'name': 'run_test', 'content': 'D:\\Repos\\Minecraft\\Forge\\SuperFactoryManager\\src\\gametest\\java\\ca\\teamdman\\sfm\\ai\\OpenDoorTest.java:54: error: no suitable method found for setDeltaMovement(Vec3,Vec3)\n            item.setDeltaMovement(Vec3.atLowerCornerOf(helper.absolutePos(pressurePlatePos).offset(0, 3, 0)), Vec3.ZERO);\n                ^\n    method Entity.setDeltaMovement(Vec3) is not applicable\n      (actual and formal argument lists differ in length)\n    method Entity.setDeltaMovement(double,double,double) is not applicable\n      (actual and formal argument lists differ in length)'}


In [45]:
response1 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        *messages,
        response0.choices[0]["message"],
        fn0,
    ],
    functions=[
        {
          "name": "run_test",
          "description": "Run the test with the agent code block substituted for the provided content. Requesting a visual observation will slow down the process through a dependency on humans.",
          "parameters": RunTest.model_json_schema()
        },
    ],
    function_call={"name": "run_test"}
)
print(response1)

{
  "id": "chatcmpl-7zen4yURBM38izZ9c10YKZ2vfsOaC",
  "object": "chat.completion",
  "created": 1694929162,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "run_test",
          "arguments": "{\n\"include_visual_observation\": false,\n\"agent_test_code\": \"item.setPos(Vec3.atCenterOf(helper.absolutePos(pressurePlatePos).offset(0, 0, 0)));\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 914,
    "completion_tokens": 44,
    "total_tokens": 958
  }
}


In [46]:
if response1.choices[0]["message"]["content"] is not None:
    print(response1.choices[0]["message"]["content"])
if "function_call" in response1.choices[0]["message"]:
    run = RunTest(**json.loads(response1.choices[0]["message"]["function_call"]["arguments"]))
    write_test(with_agent_content(read_test(), indent_text(run.agent_test_code, 12)))
    success, build_output = hot_reload()
    if not success:
        print(build_output)

[Hot Reload] Focusing IntelliJ
Error focusing window: 
[Hot Reload] Trigger hot reload
[Hot Reload] Wait for build to finish
[Hot Reload] Copy build output
[Hot Reload] Build output succeeded: False
D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\OpenDoorTest.java:54: error: no suitable method found for setDeltaMovement(Vec3,Vec3)
            item.setDeltaMovement(Vec3.atLowerCornerOf(helper.absolutePos(pressurePlatePos).offset(0, 3, 0)), Vec3.ZERO);
                ^
    method Entity.setDeltaMovement(Vec3) is not applicable
      (actual and formal argument lists differ in length)
    method Entity.setDeltaMovement(double,double,double) is not applicable
      (actual and formal argument lists differ in length)


In [30]:
if success:
    test_output = run_test()

[Run Test] Running test, ensure you ran `/sfm_ai listen` in Minecraft


In [31]:
fn1 = {
    "role": "function",
    "name": "run_test",
    "content": build_output if not success else test_output,
}

In [32]:
response2 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        *messages,
        response0.choices[0]["message"],
        fn0,
        response1.choices[0]["message"],
        fn1
    ],
    functions=[
        {
          "name": "run_test",
          "description": "Run the test with the agent code block substituted for the provided content. Requesting a visual observation will slow down the process through a dependency on humans.",
          "parameters": RunTest.model_json_schema()
        },
    ],
)
print(response2)

{
  "id": "chatcmpl-7zel30gj3aHdyi8E3vbE0n3dpvD9H",
  "object": "chat.completion",
  "created": 1694929037,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "In the game, the pressure plate becomes \"powered\" when something touches or is on top of it, this sends a \"redstone signal\" to nearby blocks, in this case, the door, forcing it to open. \n\nThe item entity must land on the pressure plate to press it and open the door with the consequent redstone signal. Currently, the program tries to spawn the item above the door which is at position (2, 2, 1). Therefore, we need to spawn the item just above the pressure plate at position (0, 3, 1) so that it falls onto the pressure plate.\n\nLet's adjust the code accordingly.",
        "function_call": {
          "name": "run_test",
          "arguments": "{\n\"include_visual_observation\": false,\n\"agent_test_code\": \"item.setPos(Vec3.atCenterOf(helper.abso

In [34]:
if response2.choices[0]["message"]["content"] is not None:
    print(response2.choices[0]["message"]["content"])
if "function_call" in response2.choices[0]["message"]:
    run = RunTest(**json.loads(response2.choices[0]["message"]["function_call"]["arguments"]))
    write_test(with_agent_content(read_test(), indent_text(run.agent_test_code, 12)))
    success, build_output = hot_reload()
    if not success:
        print(build_output)
    if success:
        test_output = run_test()
        print("test_out", test_output)

In the game, the pressure plate becomes "powered" when something touches or is on top of it, this sends a "redstone signal" to nearby blocks, in this case, the door, forcing it to open. 

The item entity must land on the pressure plate to press it and open the door with the consequent redstone signal. Currently, the program tries to spawn the item above the door which is at position (2, 2, 1). Therefore, we need to spawn the item just above the pressure plate at position (0, 3, 1) so that it falls onto the pressure plate.

Let's adjust the code accordingly.
[Hot Reload] Focusing IntelliJ
Error focusing window: 
[Hot Reload] Trigger hot reload
[Hot Reload] Wait for build to finish
[Hot Reload] Copy build output
[Hot Reload] Build output succeeded: True
[Run Test] Running test, ensure you ran `/sfm_ai listen` in Minecraft
test_out open_door failed! Expected property open to be true, was false at 2,-58,4 (relative: 2,2,1) (t=60)


In [112]:
save_convo([
    *messages,
    response0.choices[0]["message"],
    fn0,
    response1.choices[0]["message"],
    fn1,
    response2.choices[0]["message"],
])